该内容是对simple_vit中内容的讲解，主要分为：
- 代码
    - attention
    - transformer
    - embedding
    - simple_vit
- 训练
    - train
- 测试
    - test

# 一、代码

这一部分包含：
- attention
- transformer
- embedding
- simple_vit

In [1]:
from functools import partial

import torch
from torch import nn, einsum
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from einops import rearrange, repeat
from einops.layers.torch import Rearrange, Reduce

import os
import shutil

## 1 attention

dim：输入特征的维度（embedding 的维度）。            
heads：注意力头的数量。            
dim_head：每个头的维度。             

这里self.to_qkv()等价于把输入 X向右扩展为 [X, X, X]，同时左乘拼接起来的矩阵 [[Wq], [Wk], [Wv]]，从而一次性得到 Q、K、V。
然后后面非常关键的一步：
```python
map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h = self.heads), qkv)
```
将三维的h个QKV首先分别拆分为Q、K和V，然后再将三维的Q、K和V分别拆为三维的Q_i、K_i和V_i，然后以四维的形式存储在三个变量q, k, v（相当于q = [Q_0, Q_1, ... , Q_h]，kv同理），所以 torch.matmul 这一步才能分别计算Q_i × K_i^T。

In [2]:
class Attention(nn.Module):
    def __init__(self, dim, heads = 8, dim_head = 64):
        super().__init__()
        inner_dim = dim_head * heads               # Q、K、V 的拼接维度。
        self.heads = heads                         
        self.scale = dim_head ** -0.5              # 用于缩放 QK 的点积，防止 softmax 后梯度过小或过大
        self.norm = nn.LayerNorm(dim)              # 层归一化，通常用于稳定训练。
        self.attend = nn.Softmax(dim = -1)         # 对最后一个维度（注意力得分）做 softmax，形成注意力权重（也就是对Score做softmax，得到Attention）
        self.to_qkv = nn.Linear(dim, inner_dim * 3, bias = False)
        self.to_out = nn.Linear(inner_dim, dim, bias = False)

    def forward(self, x):
        x = self.norm(x)
        qkv = self.to_qkv(x).chunk(3, dim = -1)
        q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h = self.heads), qkv)     # 这是非常关键的一步处理
        dots = torch.matmul(q, k.transpose(-1, -2)) * self.scale
        attn = self.attend(dots)
        out = torch.matmul(attn, v)
        out = rearrange(out, 'b h n d -> b n (h d)')
        return self.to_out(out)

## 2 transformer

这一部分是利用前面的注意力计算方法搭配一个前馈神经网络 FeedForward Neural Network 来进行 transformer encoder 块的构建。

### 2.1 FeedForward
              
主要目的是为了引入非线性，提高模型的表达能力。    

### 2.2 ModuleList    

torch.nn.ModuleList 是一个容器，专门用于存放多个子模块，和 nn.Sequential 不同，Sequential 是一个可执行模型，自动按顺序执行子模块，而 nn.ModuleList 是一个模块列表容器，不执行，只注册，执行逻辑自己写，就比如本模块中的前向传播部分：

```python
def forward(self, x):
    for attn, ff in self.layers:
        x = attn(x) + x
        x = ff(x) + x
    return self.norm(x)
```

Sequential 只适用于线性、固定的前向流程，但是本模型中要使用残差连接，所以要使用 ModuleList。

### 2.3 Transformer

dim：输入 token 的维度（即 embedding 维度）            
depth：Block 的层数，也就是 Transformer 的堆叠深度             
heads：注意力头的数量              
dim_head：每个注意力头的维度             
mlp_dim：FeedForward 中间层的维度（即升维后的维度）                 

In [3]:
class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Linear(hidden_dim, dim),
        )
    def forward(self, x):
        return self.net(x)

class Transformer(nn.Module):
    def __init__(self, dim, depth, heads, dim_head, mlp_dim):
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.layers = nn.ModuleList([
            nn.ModuleList([
                Attention(dim, heads=heads, dim_head=dim_head),
                FeedForward(dim, mlp_dim)
            ])
            for _ in range(depth)
        ])
    def forward(self, x):
        for attn, ff in self.layers:
            x = attn(x) + x
            x = ff(x) + x
        return self.norm(x)

## 3 embedding

这段负责图像切分、Patch 嵌入和位置编码，也就是将图像输入转为 token 表示的模块。

### 3.1 pair

如果输入是整数，就把它变成形如 (t, t) 的二元组，e.g：
pair(32) → (32, 32)
pair((16, 8)) → (16, 8)

### 3.2 posemb_sincos_2d

h、w：图像划分 patch 后的高和宽（即 patch 网格大小）                
dim：每个位置编码的维度，必须是 4 的倍数               
temperature：控制正余弦波的频率范围，默认为 10000（和原始 Transformer 中一致）             
dtype：输出的张量类型（float32）              

这里不详细介绍这个编码模式了，只需要简单了解一下即可。

在ViT原论文中，位置编码是一维的、可学习的（learnable 1D positional embedding），但是 vit-pytorch 中的 simple_vit.py 实现中，作者 lucidrains 使用了不可学习的二维正余弦位置编码（sinusoidal 2D positional encoding），原因如下：
- 不需要训练，因此泛化性好
- 对小模型更稳定
- 简洁性与泛化性优先
- 避免可学习位置编码带来的 patch 数不匹配问题

### 3.3 PatchEmbedding

image_size：输入图像尺寸（可以是 int 或 tuple）          
patch_size：每个 patch 的大小            
dim：输出 token 的维度（Transformer 的输入维度）            
channels：图像的通道数，默认为 3（RGB）              

Transformer 只能处理固定维度的 token，所以先

```python
self.rearrange = Rearrange("b c (h p1) (w p2) -> b (h w) (p1 p2 c)", p1=patch_height, p2=patch_width)
```

将h×w×c的三维图片转换成(h×w/(p^2))×(p^2×c)的二维矩阵。

```python
self.net = nn.Sequential(
    self.rearrange,
    nn.LayerNorm(patch_dim),
    nn.Linear(patch_dim, dim),
    nn.LayerNorm(dim)
)
```

用Linear将patch_dim转化为transformer可处理的dim，防止因图片大小变化而引起的变化导致无法处理。       

In [4]:
def pair(t):
    return t if isinstance(t, tuple) else (t, t)

def posemb_sincos_2d(h, w, dim, temperature: int = 10000, dtype=torch.float32):
    y, x = torch.meshgrid(torch.arange(h), torch.arange(w), indexing="ij")
    assert (dim % 4) == 0, "feature dimension must be multiple of 4 for sincos emb"
    omega = torch.arange(dim // 4) / (dim // 4 - 1)
    omega = 1.0 / (temperature ** omega)
    y = y.flatten()[:, None] * omega[None, :]
    x = x.flatten()[:, None] * omega[None, :]
    pe = torch.cat((x.sin(), x.cos(), y.sin(), y.cos()), dim=1)
    return pe.type(dtype)

class PatchEmbedding(nn.Module):
    def __init__(self, image_size, patch_size, dim, channels=3):
        super().__init__()
        image_height, image_width = pair(image_size)
        patch_height, patch_width = pair(patch_size)
        patch_dim = channels * patch_height * patch_width

        self.rearrange = Rearrange("b c (h p1) (w p2) -> b (h w) (p1 p2 c)", p1=patch_height, p2=patch_width)
        self.net = nn.Sequential(
            self.rearrange,
            nn.LayerNorm(patch_dim),
            nn.Linear(patch_dim, dim),
            nn.LayerNorm(dim)
        )

    def forward(self, x):
        return self.net(x)

## 4 simple

定义完整Simple-ViT模型，该模型中没有添加原论文中的CLS，而是在最后添加了一层平均池化，目的还是为了简洁稳定、收敛快，但是同样的，模型的能力相对来讲会弱很多。

image_size：输入图像尺寸（可为 int 或 tuple）
patch_size：patch 的高宽
num_classes：分类类别数（用于输出层）
dim：token 向量的维度，也是 Transformer 的输入维度
depth：Transformer block 的层数
heads：多头注意力的头数
dim_head：每个注意力头的维度
mlp_dim	Transformer：中前馈层的中间维度
channels：图像通道数，默认 3（RGB）

```python
image_height, image_width = pair(image_size)
patch_height, patch_width = pair(patch_size)

assert image_height % patch_height == 0 and image_width % patch_width == 0
```

确保图像尺寸可以整除 patch 尺寸，便于划分为整齐的 patch 网格

```python
self.pool = "mean"
```

用来取代CLS的平均池化

```python
x += self.pos_embedding.to(device, dtype=x.dtype)
```

中to device是为了确保位置编码和输入在同一个设备上，否则会报错

```python
self.to_latent = nn.Identity()

x = self.to_latent(x)
```

这一部分什么也没做，是为了将来接口占位置（如果想用MLP之类的进行分类，只需要将其修改即可，不用重新写forward）

```python
nn.Linear(dim, num_classes)
```

是一个全连接层，用于最后的分类。

In [5]:
class SimpleViT(nn.Module):
    def __init__(self, *, image_size, patch_size, num_classes, dim, depth, heads, mlp_dim, channels=3, dim_head=64):
        super().__init__()
        image_height, image_width = pair(image_size)
        patch_height, patch_width = pair(patch_size)

        assert image_height % patch_height == 0 and image_width % patch_width == 0, "Image dimensions must be divisible by the patch size."

        self.to_patch_embedding = PatchEmbedding(image_size, patch_size, dim, channels)

        self.pos_embedding = posemb_sincos_2d(
            h = image_height // patch_height,
            w = image_width // patch_width,
            dim = dim,
        )

        self.transformer = Transformer(dim, depth, heads, dim_head, mlp_dim)

        self.pool = "mean"
        self.to_latent = nn.Identity()
        self.linear_head = nn.Linear(dim, num_classes)

    def forward(self, img):
        device = img.device
        x = self.to_patch_embedding(img)
        x += self.pos_embedding.to(device, dtype=x.dtype)
        x = self.transformer(x)
        x = x.mean(dim=1)
        x = self.to_latent(x)
        return self.linear_head(x)

# 训练

我们使用[Tiny ImageNet](http://cs231n.stanford.edu/tiny-imagenet-200.zip)来进行训练和验证，该数据集是 ImageNet 数据集的一个精简版本，共有200个类，每个类有 500 张train、50 张val，另外还有10000张无标签的test，图像尺寸为64*64

## 准备数据集

Tiny ImageNet 中的 val 目录结构不是标准的 ImageFolder 格式，需要处理一下

其他部分正常准备即可

In [6]:
# 处理 val
def organize_val_folder(val_dir):
    val_img_dir = os.path.join(val_dir, 'images')
    with open(os.path.join(val_dir, 'val_annotations.txt')) as f:
        lines = f.readlines()

    for line in lines:
        tokens = line.strip().split('\t')
        img, label = tokens[0], tokens[1]
        label_dir = os.path.join(val_dir, label)
        os.makedirs(label_dir, exist_ok=True)
        shutil.move(os.path.join(val_img_dir, img), os.path.join(label_dir, img))

    shutil.rmtree(val_img_dir)

organize_val_folder('../../datasets/tiny-imagenet-200/val')


# 准备数据集
data_dir = '../../datasets/tiny-imagenet-200'

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

train_dataset = datasets.ImageFolder(root=f"{data_dir}/train", transform=transform)
val_dataset = datasets.ImageFolder(root=f"{data_dir}/val", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
val_loader   = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4)

## 初始化模型

In [7]:
model = SimpleViT(
    image_size=224,
    patch_size=16,
    num_classes=200,
    dim=512,
    depth=6,
    heads=8,
    mlp_dim=1024
).cuda()  # 没有GPU的可以不.cuda()

## 训练

In [9]:
writer = SummaryWriter(log_dir='../../tensorboard/simple_vit')

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=3e-4)
num_epochs = 10

for epoch in range(num_epochs):
    print("epoch:{}".format(epoch))
    # Train
    model.train()
    total, correct = 0, 0
    running_loss = 0.0
    for batch_idx, (images, labels) in enumerate(train_loader):
        print("batch_idx:{}".format(batch_idx))
        
        images, labels = images.cuda(), labels.cuda()       # 若无GPU去掉.cuda()

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = outputs.max(1)                           # 取每行最大值和索引，得到预测类别
        correct += (preds == labels).sum().item()           # 比较预测和真实标签是否一致；
        total += labels.size(0)

        # 每 step 记录一次 loss
        writer.add_scalar('Loss/train_step', loss.item(), epoch * len(train_loader) + batch_idx)

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = correct / total
    writer.add_scalar('Loss/train_epoch', epoch_loss, epoch)
    writer.add_scalar('Accuracy/train_epoch', epoch_acc, epoch)

    print(f"Epoch {epoch+1}: Train Loss = {epoch_loss:.4f}, Train Acc = {epoch_acc:.4f}")

    # Val
    model.eval()
    val_total, val_correct = 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.cuda(), labels.cuda()
            outputs = model(images)
            _, preds = outputs.max(1)
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)

    val_acc = val_correct / val_total
    writer.add_scalar('Accuracy/val_epoch', val_acc, epoch)
    print(f"Epoch {epoch+1}: Validation Accuracy = {val_acc:.4f}")

    save_path = f'../../model/simple_vit/simple_vit_epoch_{epoch+1}.pth'
    torch.save(model.state_dict(), save_path)
    print(f"Saved model checkpoint to {save_path}")

writer.close()

epoch:0
batch_idx:0
batch_idx:1
batch_idx:2
batch_idx:3
batch_idx:4
batch_idx:5
batch_idx:6
batch_idx:7
batch_idx:8
batch_idx:9
batch_idx:10
batch_idx:11
batch_idx:12
batch_idx:13
batch_idx:14
batch_idx:15
batch_idx:16
batch_idx:17
batch_idx:18
batch_idx:19
batch_idx:20
batch_idx:21
batch_idx:22
batch_idx:23
batch_idx:24
batch_idx:25
batch_idx:26
batch_idx:27
batch_idx:28
batch_idx:29
batch_idx:30
batch_idx:31
batch_idx:32
batch_idx:33
batch_idx:34
batch_idx:35
batch_idx:36
batch_idx:37
batch_idx:38
batch_idx:39
batch_idx:40
batch_idx:41
batch_idx:42
batch_idx:43
batch_idx:44
batch_idx:45
batch_idx:46
batch_idx:47
batch_idx:48
batch_idx:49
batch_idx:50
batch_idx:51
batch_idx:52
batch_idx:53
batch_idx:54
batch_idx:55
batch_idx:56
batch_idx:57
batch_idx:58
batch_idx:59
batch_idx:60
batch_idx:61
batch_idx:62
batch_idx:63
batch_idx:64
batch_idx:65
batch_idx:66
batch_idx:67
batch_idx:68
batch_idx:69
batch_idx:70
batch_idx:71
batch_idx:72
batch_idx:73
batch_idx:74
batch_idx:75
batch_idx:76
b

KeyboardInterrupt: 

# 测试

In [8]:
def test_simple_vit_output_shape():
    model = SimpleViT(
        image_size=64,
        patch_size=16,
        num_classes=10,
        dim=128,
        depth=6,
        heads=8,
        mlp_dim=256
    )
    img = torch.randn(2, 3, 64, 64)
    out = model(img)
    assert out.shape == (2, 10)